In [1]:
import pandas as pd
import re
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import ArcGIS
from tqdm import tqdm
tqdm.pandas()

In [2]:
%%time
df = pd.read_csv('padron.csv')
df['MESA'] = df['MESA'].apply(lambda x: x.replace(' ', ''))
df['DOMICILIO ELECTORAL'] = df['DOMICILIO ELECTORAL'].astype('str')
df = df.sort_values(by='DOMICILIO ELECTORAL')

Wall time: 7.98 ms


In [3]:
def formating(x):
    tipocalle = ['AV', 'PASAJE', 'CALLE']
    if re.match(r'[0-9]+ ', x):
        f1 = re.match(r'[0-9]+ ', x).group(0).strip()
        f2 = re.sub(f1, '', x, count=1).strip()
        f3 = re.match(r'[^\d]*', f2).group(0).strip()
        f4 = re.sub(f3, '', f2, count=1).strip()
        if f4!='':
            f5 = re.match(r'[0-9]+', f4).group(0).strip()
            return f1 + ' ' + f3 + ' ' + f5
        else:
            return None
    elif re.match(r'[^\d]*', x):
        f1 = re.match(r'[^\d]*', x).group(0).strip()
        f2 = re.sub(f1, '', x, count=1).strip()
        if f1 in tipocalle and f2!='':
            f3 = re.match(r'[0-9]+', f2).group(0).strip()
            f4 = re.sub(f3, '', f2, count=1).strip()           
            if f4!='' and re.match(r'[^\d]*', f4).group(0).strip():
                f5 = re.match(r'[^\d]*', f4).group(0).strip()
                f6 = re.sub(f5, '', f4, count=1).strip()
                if f6!='' and re.match(r'[0-9]+', f6).group(0).strip():
                    f7 = re.match(r'[0-9]+', f6).group(0).strip()
                    return f1 + ' ' + f3 + ' ' + f5 + ' ' + f7
                else:
                    return None
            elif f4!='' and re.match(r'[0-9]+', f4).group(0).strip():
                    f5 = re.match(r'[0-9]+', f4).group(0).strip()
                    return f1 + ' ' + f3 + ' ' + f5                
            else:
                return None
        elif f1 not in tipocalle and f2!='' and re.match(r'[0-9]+', f2).group(0).strip():
            f3 = re.match(r'[0-9]+', f2).group(0).strip()
            return f1 + ' ' + f3
        else:
            return None

In [4]:
print('Cantidad de Electores por Circunscripción')
for i in enumerate(df['CIRCUNSCRIPCIÓN'].unique()):
    print('\n'+ f'{i[0]})' + ' ' + f'{i[1]}: ' + str(len(df[df['CIRCUNSCRIPCIÓN']==f'{i[1]}'])))

Cantidad de Electores por Circunscripción

0) EL PRINCIPAL: 77

1) PIRQUE: 178


In [5]:
comuna = input('Indique el nombre de la comuna: ').upper()
circuns = df['CIRCUNSCRIPCIÓN'].unique()[int(input('Indique el número de la circunscripción: '))]

Indique el nombre de la comuna: pirque
Indique el número de la circunscripción: 1


In [6]:
df_Final = df[df['CIRCUNSCRIPCIÓN']==circuns]
df_Final = df_Final.sort_values(by='DOMICILIO ELECTORAL')

#FORMATEO DE DIRECCIONES
df_Final['DOMICILIO ELECTORAL IMP'] = df_Final['DOMICILIO ELECTORAL'].apply(lambda x: formating(x))

df_Final['DOMICILIO ELECTORAL IMP'] = df_Final['DOMICILIO ELECTORAL IMP'].astype('str')
df_Final.loc[df_Final['DOMICILIO ELECTORAL IMP'] != 'None', 'DOMICILIO ELECTORAL FINAL'] = df_Final.loc[df_Final['DOMICILIO ELECTORAL IMP']!= 'None', 'DOMICILIO ELECTORAL IMP'] + ', ' + comuna + ', REGION METROPOLITANA DE SANTIAGO'
df_Final['DOMICILIO ELECTORAL FINAL'] = df_Final['DOMICILIO ELECTORAL FINAL'].astype('str')

print('Total de Electores: ' + str(len(df_Final['DOMICILIO ELECTORAL FINAL'] + '\n')))
print('Direcciones únicas: ' + str(len(df_Final['DOMICILIO ELECTORAL FINAL'].unique())))

Total de Electores: 178
Direcciones únicas: 118


In [7]:
df_Geopy = pd.DataFrame(df_Final['DOMICILIO ELECTORAL FINAL'].unique(), columns={'DOMICILIO ELECTORAL FINAL'})
df_Geopy = df_Geopy.reset_index(drop=True)

geolocator = ArcGIS(timeout=15)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_Geopy['GEOPY'] = df_Geopy['DOMICILIO ELECTORAL FINAL'].progress_apply(geocode)

df_Geopy2 = df_Geopy[df_Geopy['GEOPY'].notnull()]
df_Geopy2['LATITUD'] = df_Geopy2['GEOPY'].apply(lambda x: x.latitude)
df_Geopy2['LONGITUD'] = df_Geopy2['GEOPY'].apply(lambda x: x.longitude)

df_Final2 = df_Final.merge(df_Geopy2, how='inner', left_on=['DOMICILIO ELECTORAL FINAL'], right_on=['DOMICILIO ELECTORAL FINAL'])
df_Final2.to_csv(f'geomap_{comuna}_{circuns}.csv', index = False)

100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [01:58<00:00,  1.00s/it]


In [8]:
df_Final2

,NOMBRE,C.IDENTIDAD,SEXO,DOMICILIO ELECTORAL,CIRCUNSCRIPCIÓN,MESA,PUEBLO INDÍGENA,DOMICILIO ELECTORAL IMP,DOMICILIO ELECTORAL FINAL,GEOPY,LATITUD,LONGITUD
0,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,VARON,ALICIA VALDES DE BALMACEDA CS 4,PIRQUE,27,NaN,ALICIA VALDES DE BALMACEDA CS 4,"ALICIA VALDES DE BALMACEDA CS 4, PIRQUE, REGIO...","(Balmaceda, Santiago, Región Metropolitana de ...",-33.431880,-70.670820
1,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,VARON,ALICIA VALDEZ 0,PIRQUE,37,NaN,ALICIA VALDEZ 0,"ALICIA VALDEZ 0, PIRQUE, REGION METROPOLITANA ...","(Circunvalación Maximo Valdes, Pirque, Cordill...",-33.627398,-70.544394
2,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,VARON,ARAUCARIA 690 VILLORRIO EL OLIVAR,PIRQUE,26,NaN,ARAUCARIA 690,"ARAUCARIA 690, PIRQUE, REGION METROPOLITANA DE...","(Calle La Araucaria 690, Pirque, Cordillera, R...",-33.707010,-70.557830
3,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,VARON,AV ALCALDE HERNAN PRIETO 22 C,PIRQUE,35,NaN,AV ALCALDE HERNAN PRIETO 22,"AV ALCALDE HERNAN PRIETO 22, PIRQUE, REGION ME...","(Avenida Alcalde Hernán Prieto 22, Pirque, Cor...",-33.656584,-70.586420
4,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,VARON,AV ALCALDE HERNAN PRIETO 22 C,PIRQUE,21M,NaN,AV ALCALDE HERNAN PRIETO 22,"AV ALCALDE HERNAN PRIETO 22, PIRQUE, REGION ME...","(Avenida Alcalde Hernán Prieto 22, Pirque, Cor...",-33.656584,-70.586420
...,...,...,...,...,...,...,...,...,...,...,...,...
173,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,MUJER,VIRGINIA SUBERCASEAUX 4000,PIRQUE,36,NaN,VIRGINIA SUBERCASEAUX 4000,"VIRGINIA SUBERCASEAUX 4000, PIRQUE, REGION MET...","(Avenida Virginia Subercaseaux, Pirque, Cordil...",-33.646110,-70.631785
174,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,VARON,VIRGINIA SUBERCASEAUX 4000 PARCELA 31,PIRQUE,19V,NaN,VIRGINIA SUBERCASEAUX 4000,"VIRGINIA SUBERCASEAUX 4000, PIRQUE, REGION MET...","(Avenida Virginia Subercaseaux, Pirque, Cordil...",-33.646110,-70.631785
175,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,MUJER,VIRGINIA SUBERCASEAUX 4000 PARCELA 31,PIRQUE,22M,NaN,VIRGINIA SUBERCASEAUX 4000,"VIRGINIA SUBERCASEAUX 4000, PIRQUE, REGION MET...","(Avenida Virginia Subercaseaux, Pirque, Cordil...",-33.646110,-70.631785
176,NNNNNNNN NNNNNNNN AAAAAAAA AAAAAAAA,00.000.000-K,MUJER,VIRGINIA SUBERCASEUX 1570 PARCELA-62,PIRQUE,14V,NaN,VIRGINIA SUBERCASEUX 1570,"VIRGINIA SUBERCASEUX 1570, PIRQUE, REGION METR...","(Avenida Virginia Subercaseaux, Pirque, Cordil...",-33.646110,-70.631785
